In [3]:
%matplotlib inline
from pymongo import MongoClient
import pandas as pd
from tqdm import *
import pandas as pd
import pylab as p
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import median_absolute_error, mean_squared_error, explained_variance_score, mean_absolute_error, r2_score
from sklearn.cross_validation import ShuffleSplit, train_test_split
import scipy
import random
import time
from multiprocessing import Pool
from scipy.stats import boxcox
import matplotlib
import bayeslite as bl
matplotlib.style.use('ggplot')

In [2]:
client = MongoClient()
db = client.nba
cursor = db.games.find()
games = []
teams = []
players = []

def str_to_int(val):
    if isinstance(val, int):
        return val
    if len(val.strip()) == 0:
        return 0
    else:
        return int(val.strip())

def str_to_float(val):
    if isinstance(val, float):
        return val
    if len(val.strip()) == 0:
        return 0.0
    else:
        return float(val.strip())    
    
for game_id, document in enumerate(tqdm(cursor)):
    game_dict = {"date":document['date'], 'game_id':game_id}
    games.append(game_dict)
    for team_index, team in enumerate(document['teams']):
        team_dict = {}
        team_dict['game_id'] = game_id
        team_dict['team_index'] = team_index
        team_dict['abbreviation'] = team['abbreviation']
        team_dict['home'] = team['home']
        team_dict['score'] = int(team['score'])
        team_dict['name'] = team['name']
        team_dict['won'] = team['won']
        teams.append(team_dict)
    for team_index, box in enumerate(document['box']):
        for player in box['players']:
            players_dict = {}
            players_dict['game_id'] = game_id
            players_dict['team_index'] = team_index
            players_dict['player'] = player['player']
            players_dict['ft_pct'] = str_to_float(player['ft_pct'])
            players_dict['fg3a'] = str_to_int(player['fg3a'])
            players_dict['trb'] = str_to_int(player['trb'])
            players_dict['ft'] = str_to_int(player['ft'])
            players_dict['ast'] = str_to_int(player['ast'])
            players_dict['stl'] = str_to_int(player['stl'])
            players_dict['pts'] = str_to_int(player['pts'])
            players_dict['fg_pct'] = str_to_float(player['fg_pct'])
            players_dict['tov'] = str_to_int(player['tov'])
            players_dict['fg3'] = str_to_int(player['fg3'])
            players_dict['fta'] = str_to_int(player['fta'])
            players_dict['fg3_pct'] = str_to_float(player['fg3_pct'])
            players_dict['pf'] = str_to_int(player['pf'])
            players_dict['mp'] = str_to_int(player['mp'].split(":")[0])*60 + str_to_int(player['mp'].split(":")[1])
            players_dict['blk'] = str_to_int(player['blk'])
            players_dict['fg'] = str_to_int(player['fg'])
            players_dict['drb'] = str_to_int(player['drb'])
            players_dict['orb'] = str_to_int(player['orb'])
            players_dict['fga'] = str_to_int(player['fga'])
            players.append(players_dict)

3804it [00:01, 2425.09it/s]Process PoolWorker-7:
Process PoolWorker-1:
Process PoolWorker-8:
Process PoolWorker-4:
Process PoolWorker-2:
Process PoolWorker-6:
Process PoolWorker-3:
Process PoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in 

KeyboardInterrupt: 

In [ ]:
players_df = pd.DataFrame(players)
players_df = players_df.set_index(['game_id'])
games_df = pd.DataFrame(games)
teams_df = pd.DataFrame(teams)
players_df = players_df.join(games_df)
players_df = players_df.set_index(['game_id', 'team_index'])
teams_df = teams_df.set_index(['game_id', 'team_index'])
players_df = players_df.join(teams_df)

In [ ]:
def draft_kings_score(row):
    score = (row['pts']*1.0) + (row['fg3']*0.5) +  (row['trb']*1.25) + (row['ast']*1.5) 
    score += (row['stl']*2.0) + (row['blk']*2.0) + (row['tov']*-0.5)
    doubles = 0
    if row['pts'] > 10:
        doubles += 1
    if row['trb'] > 10:
        doubles += 1
    if row['ast'] > 10:
        doubles += 1
    if row['blk'] > 10:
        doubles += 1
    if row['stl'] > 10:
        doubles += 1
    # double double
    if doubles == 2:
        score += 1.5
    # triple double
    if doubles == 3:
        score + 3
    return score    

players_df['dk_fpts'] = players_df.apply(lambda row:  draft_kings_score(row), axis=1)
players_df['year'] = players_df['date'].map(lambda x: x.year)

In [ ]:
player_min_dates = {}
groups = players_df.groupby(by=['player'])
for name, group in groups:
    player_min_dates[name] = group['date'].min()

In [ ]:
def get_player_season(row):
    date = row['date']
    min_date = player_min_dates[row['player']]
    diff = date - min_date
    return diff.total_seconds()/3600./24./365.

players_df['years_in_league'] = players_df.apply(lambda row: get_player_season(row), axis=1)

In [ ]:
plt.hist(players_df['dk_fpts'].values, bins=25)

In [ ]:
players_df = players_df.reset_index()
groups = players_df.groupby(by=['player'])

In [ ]:
def get_flattened_columns(prefix, pivot_table):    
    cols = []
    col_dict = {}
    for i, (col, gid) in enumerate(pivot_table.columns.tolist()):
        if gid not in col_dict:
            col_dict[gid] = str(i)
        cols.append(prefix+col_dict[gid]+"_"+col)
    return cols

features = pd.DataFrame()

for name, group in tqdm(groups):
    group = group.sort(['game_id'], ascending=[False])
    try:
        for idx, row in group.iterrows():
            current_fpts = row['dk_fpts']
    #         print row['game_id'], current_fpts
            groups_lt = group[group['game_id'] < row['game_id']][0:10]
    #         print groups_lt
            if len(groups_lt) >= 10:
                player_games_pivot = groups_lt.pivot(index='player', columns='game_id')
                cols = player_games_pivot.columns.tolist()
                cols = sorted(cols, reverse=True)
                player_games_pivot = player_games_pivot[cols]
                player_games_pivot['current_fpts'] = current_fpts
                pivot = player_games_pivot[['current_fpts', 'dk_fpts', 'year', 'years_in_league']]
    #             print pivot
                cols = get_flattened_columns("", pivot)
                pivot.columns = cols
#                 print pivot
#                 break
                features = pd.concat([features, pivot])
    except Exception as e:
        print e
#             print features
#     break

In [ ]:
features

In [ ]:
features.to_csv("/Users/jostheimer/workspace/kaggle/data/nba/features_10.csv")

In [ ]:
features_save = features.copy(deep=True)

In [4]:
features = pd.read_csv("/Users/jostheimer/workspace/kaggle/data/nba/features_10.csv", index_col=[0])

In [ ]:
if '0_current_fpts' in features.columns:
    targets = features['0_current_fpts']
    features = features.drop(['0_current_fpts'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, random_state=42)

# clf = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

cfr = ExtraTreesRegressor(
    oob_score = True, bootstrap=True, verbose = 1, max_features = len(features.columns), n_estimators = 10000, min_samples_leaf = 1,
    n_jobs=8,
    random_state=0,
)
cfr.fit(x_train, y_train)



In [ ]:
predictions = cfr.predict(x_test)
x_test['prediction'] = predictions
# predictions = inverse_box_cox(predictions,lambda_, min_target)
plt.scatter(y_test, predictions)
plt.show()
print "mean_absolute_error", mean_absolute_error(y_test, predictions)
print "mean_squared_error", mean_squared_error(y_test, predictions)
print "explained_variance", explained_variance_score(y_test, predictions)
print "median_absolute_error", median_absolute_error(y_test, predictions)
print "r2_score", r2_score(y_test, predictions)

In [ ]:
print "Features sorted by their score:"
sorted_features =  sorted(zip(map(lambda x: round(x, 4), cfr.feature_importances_), x_train.columns), 
             reverse=True)
for sorted_feat in sorted_features:
    print sorted_feat